In [45]:
from pulp import *
import pandas as pd
from itertools import product

In [46]:
df_cases = pd.read_csv("cases.csv")
df_sessions = pd.read_csv("sessions.csv")

In [47]:
# Model data

# list of case IDs
cases = df_cases["CaseID"].tolist()
# list of session IDs
sessions = df_sessions["SessionID"].tolist()
# all possible combinations of cases and sessions
tasks = list(product(cases, sessions))
# expected duration for each operation
case_duration = pd.Series(df_cases["Expected Duration"].values, index=df_cases["CaseID"]).to_dict()
# duration of each OR room availability
session_duration = pd.Series(df_sessions["Duration"].values, index=df_sessions["SessionID"]).to_dict()
# session start times
df_sessions.loc[:, "Start"] = pd.to_timedelta(df_sessions["Start"])
df_sessions.loc[:, "Start"] = df_sessions["Start"].dt.total_seconds() / 60
session_start_time = pd.Series(df_sessions["Start"].values, index=df_sessions["SessionID"]).to_dict()
# case deadline
df_cases.loc[:, "TargetDeadline"] = pd.to_datetime(df_cases["TargetDeadline"], format="%d/%m/%Y")
df_cases.loc[:, "TargetDeadline"] = df_cases["TargetDeadline"].apply(lambda date: date.toordinal()) # Gregorian ordinal for the given DateTime object.
case_deadlines = pd.Series(df_cases["TargetDeadline"].values, index=df_cases["CaseID"]).to_dict()
# Session dates
df_sessions.loc[:, "Date"] = pd.to_datetime(df_sessions["Date"], format="%d/%m/%Y")
df_sessions.loc[:, "Date"] = df_sessions["Date"].apply(lambda date: date.toordinal()) # Gregorian ordinal for the given DateTime object.
session_dates = pd.Series(df_sessions["Date"].values, index=df_sessions["SessionID"]).to_dict()

In [48]:
model = LpProblem("MaximizeUtilizationTime", LpMaximize) #spaces not permitted on name

In [49]:
#big M
M = 900000000

# Upper bound (minutes in a day)
ub = 1440 # q1

# Upper bound of session utilization set to 85%
max_util = 0.9 #q2 (we could change this?)

# Binary flag, 1 if case is assigned to session, 0 o/w
A = LpVariable('session_assigned', cat='Binary')

# Start time of case
#B = LpVariable.dicts("case_start_time", ((i, j) for i in I for j in J), lowBound = 0, cat = 'Continuous')
B = LpVariable('case_start_time', lowBound=0, upBound=ub)

# Session utilization
C = LpVariable('utilization', lowBound=0, upBound=max_util)

In [50]:
obj_func = lpSum(C)
model += obj_func

In [51]:
# The start time of a case must be after the start time of the session it is assigned to
# should only be valid for the session assigned to the case, not for the rest of the sessions
#c1 = session_start_time - (1 - A)*M <= B

for i in session_start_time:
    model += B <= i - (1 - A)*M
#c2 = session_start_time + session_duration + (1 - A)*M >= B + case_duration
#c3 = 

In [52]:
model

MaximizeUtilizationTime:
MAXIMIZE
1*utilization + 0
SUBJECT TO
_C1: case_start_time - 900000000 session_assigned <= -899998999

_C2: case_start_time - 900000000 session_assigned <= -899998998

_C3: case_start_time - 900000000 session_assigned <= -899998997

_C4: case_start_time - 900000000 session_assigned <= -899998996

VARIABLES
case_start_time <= 1440 Continuous
0 <= session_assigned <= 1 Integer
utilization <= 0.9 Continuous

In [53]:
df = pd.read_csv('sessions.csv')

In [54]:
df.head()

,SessionID,Date,Start,End,Duration,ConsultantID,Specialty
0,1001,03/06/2020,08:30:00,18:00:00,570,11,Ophthalmology
1,1002,10/06/2020,08:30:00,18:00:00,570,11,Ophthalmology
2,1003,17/06/2020,08:30:00,18:00:00,570,11,Ophthalmology
3,1004,25/06/2020,08:30:00,13:00:00,270,11,Ophthalmology
